## BCH pediatrics trends: query and plot
This notebook includes the SQL code used to extract the information regarding pediatric patients with a mental ICD-10 code condition (the full list can be found here: https://github.com/hms-dbmi/psy_peds_trends/blob/main/data/pediatric%20psychiatric%20ICD-10%20codes_adjusted.csv).

The inclusion criteria is:
- inpatients
- Sars-Cov-2 PCR  positive or negative (no filter by covid-19 condition, just PCR test done at BCH)
- filtered by age (11-17)
- 21st April 2020 - Up to date 
- The ICD-10 code from the list as Primary diagnosis (modifier → ICD:P) 

In [ ]:
#CREATE TABLE ag440_mental_health_primary
#AS SELECT obs.patient_num, obs.concept_cd, obs.start_date, pat.birth_date, floor(months_between(obs.start_date, pat.birth_date) / 12) as age, 
#         peds_codes.mental_health_disorder_group,  peds_codes.description, obs.modifier_cd, vis.inout_cd
#FROM observation_fact obs
#INNER JOIN patient_dimension pat ON obs.patient_num=pat.patient_num
#INNER JOIN visit_dimension vis ON obs.encounter_num= vis.encounter_num
#INNER JOIN ASE_PEDS_ICD_CODES peds_codes ON obs.concept_cd=peds_codes.ICD10_CODE
#WHERE floor(months_between(obs.start_date, pat.birth_date) / 12) > 10 
#      AND floor(months_between(obs.start_date, pat.birth_date) / 12) < 18 
#      AND lower(vis.inout_cd) = 'inpatient' 
#      AND obs.modifier_cd = 'ICD:P';

The output is a table that contains:
- patient identifier
- concept code (ICD-10 code)
- start date (day-month-year, 30-SEP-20)
- birth date (day-month-year, 30-SEP-03)
- age (in years)
- the mentah health disorder group (as described in the reference table)
- the description of each diagnosis
- modifier (ICD:P as primary diagnosis, ICD:S as secondary diagnosis if secondary diagnosis included)
- inout_cd (that includes the information for that patient and time as inpatient in this case)

### Barplot representation
Using as input the previous table we generate a barplot representing the number of cases per week starting the 21st April 2021 up to date. 

In [ ]:
#read the file with the dataset
output <- read.delim("mentalHealthPrimary.dsv")

In [ ]:
# filter by date, to include only those starting the 21st April 2021
output$date <- sapply(strsplit( output$START_DATE, " "), '[', 1)
output$date <- as.Date( output$date, "%d-%B-%y")
output <- output[ output$date > "2020-04-21",]

#breaks by week
output$weeks <- cut(output[,"date"], breaks="week")

#select the patient identifier and the week and remove potential duplicates 
subset <- unique(output[ , c("PATIENT_NUM", "weeks")])
length(unique(subset$PATIENT_NUM))
summary( subset$weeks )

In [ ]:
#load the libraries
library(dplyr)
library(ggplot2)

#estimate the number of patieents per week
byWeek <- subset %>% group_by(weeks) %>% tally()
byWeek$year <-sapply(strsplit( as.character(byWeek$weeks), "-"), '[', 1) 
byWeek$period <- "post"

#represent it as a barplot
ggplot(data= byWeek, aes(x= as.Date(weeks), y= n, fill= period )) +
  geom_bar(stat="identity")+
  ylab("counts") + xlab("Week")+ theme_bw()+
  geom_smooth(method = "lm", se= TRUE, aes(colour=period), colour= "blue", size = 0.5)+ 
  scale_fill_manual(values=c('#00BFC4', '#F8766D'))

We can also visualize the trends for specific menth health disorders groups.

In [ ]:
#### by group category
subsetByCategory <- unique(output[ , c("PATIENT_NUM", "weeks", "MENTAL_HEALTH_DISORDER_GROUP")])
colnames( subsetByCategory ) <- c("patient_num", "weeks", "category")
byWeekCategory <- subsetByCategory %>% group_by(weeks, category) %>% tally()
byWeekCategory$period <- "post"
ggplot(data= byWeekCategory, aes(x= as.Date(weeks), y= n, fill= period, group=category )) +
  geom_bar(stat="identity")+
  ylab("counts") + xlab("Week")+ theme_bw()+
  geom_smooth(method = "lm", se= TRUE, aes(colour=period), colour= "blue", size = 0.5)+ 
  scale_fill_manual(values=c('#00BFC4', '#F8766D'))+
  facet_grid(. ~ category)


In [ ]:
selection <- byWeekCategory[ byWeekCategory$category %in% 
                               c("Anxiety Disorders", "Depressive Disorders",
                                 "Suicide or Self-Injury", 
                                 "Obsessive-Compulsive and Related Disorders", 
                                 "Substance-Related and Addictive Disorders", "Feeding and Eating Disorders"), ]


selection%>% ggplot() +
  geom_line(aes(x = as.Date(weeks), y = n,  group=category), color = "steelblue", size=0.5)+
  scale_x_date(name = "", 
               breaks = seq.Date(as.Date("2020-04-20"), 
                                 as.Date("2021-04-12"), by = "1 week"), 
               date_labels = "%b-%U")+ 
  scale_y_continuous(name = "number patients")+ 
  facet_wrap(facets = "category", scales = "free",
             labeller = label_wrap_gen(width=40)) +
  labs(x = "calendar weeks", y = "number patients consulting for condition")+
  theme_bw()+
  theme(strip.text.x = element_text(size = 7), text = element_text(size=5), 
        axis.text.x = element_text(angle = 45, vjust = 1, hjust=1))

In [ ]:
### bars
selection %>% ggplot(aes(x = as.Date(weeks), y = n)) +
  geom_bar( stat= 'identity', fill= 'steelblue')+
  scale_x_date(name = "", 
               breaks = seq.Date(as.Date("2020-04-20"), 
                                 as.Date("2021-04-12"), by = "1 week"), 
               date_labels = "%b-%U")+ 
  scale_y_continuous(name = "number patients")+ 
  facet_wrap(facets = "category", scales = "free",
             labeller = label_wrap_gen(width=40)) +
  labs(x = "calendar weeks", y = "number patients consulting for condition")+
  theme_bw()+
  theme(strip.text.x = element_text(size = 7), text = element_text(size=5), 
        axis.text.x = element_text(angle = 45, vjust = 1, hjust=1))



In [ ]:
sessionInfo()